In [43]:
% pylab inline

Populating the interactive namespace from numpy and matplotlib


/home/cscorley/envs/triage/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['f', 'info']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
from __future__ import print_function
import os.path
import pandas
import src
import sklearn
import os
import scipy
import scipy.stats
import csv

In [3]:
def fake(*args, **kwargs):
    print('Fake called with', str(args), str(kwargs))
    sys.exit(1)

# fake out the create_model so we don't accidentally attempt to create data
src.common.create_model = fake

In [4]:
print(os.getcwd())
if os.getcwd().endswith('notebooks'):
    os.chdir('..')
print(os.getcwd())

/home/cscorley/git/triage/notebooks
/home/cscorley/git/triage


In [5]:
args = dict(level='file', force=False, model='lda', source=['release', 'changeset', 'temporal'], random_seed_value=1)

model_config, model_config_string = src.main.get_default_model_config(args)
args.update({'model_config': model_config, 'model_config_string': model_config_string})

changeset_config, changeset_config_string = src.main.get_default_changeset_config()
args.update({'changeset_config': changeset_config, 'changeset_config_string': changeset_config_string})

projects = src.common.load_projects(args)
projects

[Project(name='tika', printable_name='Tika v1.8', version='v1.8', ref='refs/tags/1.8', data_path='data/tika/', full_path='data/tika/v1.8/', src_path='data/tika/v1.8/src/', changeset_config_string=u'True-True-False-True', source=['release', 'changeset', 'temporal'], model_config_string=u'seed1-batch-0.002-0.5-0.002-1000-1000-500-1.0-1', force=False, level='file', model_config={'passes': 1, 'eta': 0.002, 'num_topics': 500, 'iterations': 1000, 'decay': 0.5, 'algorithm': 'batch', 'alpha': 0.002, 'offset': 1.0, 'max_bound_iterations': 1000}, model='lda', random_seed_value=1, changeset_config={'include_removals': True, 'include_context': True, 'include_message': False, 'include_additions': True}),
 Project(name='pig', printable_name='Pig v0.14.0', version='v0.14.0', ref='refs/tags/release-0.14.0', data_path='data/pig/', full_path='data/pig/v0.14.0/', src_path='data/pig/v0.14.0/src/', changeset_config_string=u'True-True-False-True', source=['release', 'changeset', 'temporal'], model_config_st

In [6]:
import dulwich
import dulwich.patch
import io
from StringIO import StringIO

In [65]:
def get_diff(repo, changeset):
    """ Return a text representing a `git diff` for the files in the
    changeset.

    """
    patch_file = StringIO()
    dulwich.patch.write_object_diff(patch_file,
                                    repo.object_store,
                                    changeset.old, changeset.new)
    return patch_file.getvalue()

def walk_changes(repo):
    """ Returns one file change at a time, not the entire diff.

    """

    for walk_entry in repo.get_walker(reverse=True):
        commit = walk_entry.commit
        
        for change in get_changes(repo, commit):
            yield change

def get_changes(repo, commit):
        # initial revision, has no parent
        if len(commit.parents) == 0:
            for changes in dulwich.diff_tree.tree_changes(
                    repo.object_store, None, commit.tree
            ):
                diff = get_diff(repo, changes)

                yield commit, None, diff, get_path(changes)

        for parent in commit.parents:
            # do I need to know the parent id?
            try:
                for changes in dulwich.diff_tree.tree_changes(
                    repo.object_store, repo[parent].tree, commit.tree, want_unchanged=False
                ):
                    diff = get_diff(repo, changes)
                    yield commit, parent, diff, get_path(changes)
            except KeyError as e:
                print("skipping commit:", commit, ", parent:", parent, ", exception:", e)
        

def get_path(change):
    path = '/dev/null'
    if change.old.path and change.new.path:
        path = change.new.path
    elif change.old.path:
        path = change.old.path
    elif change.new.path:
        path = change.new.path
    return path


In [52]:
import re

In [53]:
unified = re.compile(r'^[+ -].*')
context = re.compile(r'^ .*')
addition = re.compile(r'^\+.*')
removal = re.compile(r'^-.*')

In [54]:
def get_commit_info(project):
    commit2linecount = dict()
    for commit, p, d, fname in walk_changes(src.common.load_repos(project)[0]):

        diff_lines = filter(lambda x: unified.match(x),
                                    d.splitlines())
        if len(diff_lines) < 2:
            # useful for not worrying with binary files
            a, r, c = 0, 0, 0
        else:
            # sanity?
            assert diff_lines[0].startswith('--- '), diff_lines[0]
            assert diff_lines[1].startswith('+++ '), diff_lines[1]
            # parent_fn = diff_lines[0][4:]
            # commit_fn = diff_lines[1][4:]

            lines = diff_lines[2:]  # chop off file names hashtag rebel
            a = len(filter(lambda x: addition.match(x), lines))
            r = len(filter(lambda x: removal.match(x), lines))
            c = len(filter(lambda x: context.match(x), lines))
        m = len(commit.message.splitlines())

        if commit.id not in commit2linecount:
            commit2linecount[commit.id] = dict()

        commit2linecount[commit.id][fname] = (a, r, c, m)
    return commit2linecount

In [55]:
def get_commit_info1(repo, commit):
    commit2linecount = dict()
    for commit, p, d, fname in get_changes(repo, commit):

        diff_lines = filter(lambda x: unified.match(x),
                                    d.splitlines())
        if len(diff_lines) < 2:
            # useful for not worrying with binary files
            a, r, c = 0, 0, 0
        else:
            # sanity?
            assert diff_lines[0].startswith('--- '), diff_lines[0]
            assert diff_lines[1].startswith('+++ '), diff_lines[1]
            # parent_fn = diff_lines[0][4:]
            # commit_fn = diff_lines[1][4:]

            lines = diff_lines[2:]  # chop off file names hashtag rebel
            a = len(filter(lambda x: addition.match(x), lines))
            r = len(filter(lambda x: removal.match(x), lines))
            c = len(filter(lambda x: context.match(x), lines))
        m = len(commit.message.splitlines())

        commit2linecount[fname] = (a, r, c, m)
    return commit2linecount

In [72]:
for project in projects:
    ids = src.common.load_ids(project)
    issue2git, git2issue = src.common.load_issue2git(project, ids, filter_ids=True)
    goldset = src.goldsets.load_goldset(project)
    #commit2linecount = get_commit_info(project)
    repo = src.common.load_repos(project)[0]
    
    with open(os.path.join(project.full_path, 'changes-file-goldset.csv'), 'w') as f:
        w =  csv.writer(f)
        w.writerow(['sha', 'issues', 'change_type', 'name', 'additions', 'removals', 'context', 'message'])
        for sha, changes in goldset.items():
            commit, change_list = changes
            info = get_commit_info1(repo, commit)
            for change_type, name in change_list:
                if sha in git2issue:
                    issues = set(git2issue[sha])
                    w.writerow([sha, ';'.join(issues), change_type, name] + list(info[name]))

    issue2git, git2issue = src.common.load_issue2git(project, ids, filter_ids=False)
    
    with open(os.path.join(project.data_path, 'changes-file-issues.csv'), 'w') as f:
        w =  csv.writer(f)
        w.writerow(['sha', 'issues', 'change_type', 'name', 'additions', 'removals', 'context', 'message'])
        for sha, changes in goldset.items():
            commit, change_list = changes
            info = get_commit_info1(repo, commit)
            for change_type, name in change_list:
                if sha in git2issue:
                    issues = set(git2issue[sha])
                    w.writerow([sha, ';'.join(issues), change_type, name]+ list(info[name]))
                    
                        
    with open(os.path.join(project.data_path, 'changes-file-full.csv'), 'w') as f:
        w =  csv.writer(f)
        w.writerow(['sha', 'change_type', 'name', 'additions', 'removals', 'context', 'message'])
        for sha, changes in goldset.items():
            commit, change_list = changes
            info = get_commit_info1(repo, commit)
            for change_type, name in change_list:
                try:
                    w.writerow([sha, change_type, name]  + list(info[name]))
                except KeyError as e:
                    print("skipping commit:", commit, ", name:", name, ", exception:", e)
                    

skipping commit: commit b830f4e88c991d67a84ed883c6136989a54c2556
tree dac06a675491b4f71805a4384c22e3456632122d
parent d17c46fc79da130e3738baaa74af9c99f1e234f2
author Sijie Guo <sijie@apache.org> 1423818911 -0800
committer Sijie Guo <sijie@apache.org> 1423818911 -0800

    BookKeeper Release 4.3.1

 , parent: d17c46fc79da130e3738baaa74af9c99f1e234f2 , exception: 'dac06a675491b4f71805a4384c22e3456632122d'
skipping commit: commit b830f4e88c991d67a84ed883c6136989a54c2556
tree dac06a675491b4f71805a4384c22e3456632122d
parent d17c46fc79da130e3738baaa74af9c99f1e234f2
author Sijie Guo <sijie@apache.org> 1423818911 -0800
committer Sijie Guo <sijie@apache.org> 1423818911 -0800

    BookKeeper Release 4.3.1

 , parent: d17c46fc79da130e3738baaa74af9c99f1e234f2 , exception: 'dac06a675491b4f71805a4384c22e3456632122d'
skipping commit: commit b830f4e88c991d67a84ed883c6136989a54c2556
tree dac06a675491b4f71805a4384c22e3456632122d
parent d17c46fc79da130e3738baaa74af9c99f1e234f2
author Sijie Guo <sijie@apa

In [46]:
sha, project

('b830f4e88c991d67a84ed883c6136989a54c2556',
 Project(name='bookkeeper', printable_name='BookKeeper v4.3.0', version='v4.3.0', ref='refs/tags/release-4.3.0', data_path='data/bookkeeper/', full_path='data/bookkeeper/v4.3.0/', src_path='data/bookkeeper/v4.3.0/src/', changeset_config_string=u'True-True-False-True', source=['release', 'changeset', 'temporal'], model_config_string=u'seed1-batch-0.002-0.5-0.002-1000-1000-500-1.0-1', force=False, level='file', model_config={'passes': 1, 'eta': 0.002, 'num_topics': 500, 'iterations': 1000, 'decay': 0.5, 'algorithm': 'batch', 'alpha': 0.002, 'offset': 1.0, 'max_bound_iterations': 1000}, model='lda', random_seed_value=1, changeset_config={'include_removals': True, 'include_context': True, 'include_message': False, 'include_additions': True}))

In [47]:
commit

<Commit b830f4e88c991d67a84ed883c6136989a54c2556>

In [48]:
name

'pom.xml'

In [49]:
changes

(<Commit b830f4e88c991d67a84ed883c6136989a54c2556>,
 [['M', 'bookkeeper-benchmark/pom.xml'],
  ['M', 'bookkeeper-server/pom.xml'],
  ['M', 'bookkeeper-stats/pom.xml'],
  ['M', 'bookkeeper-stats-providers/codahale-metrics-provider/pom.xml'],
  ['M', 'bookkeeper-stats-providers/pom.xml'],
  ['M', 'bookkeeper-stats-providers/twitter-ostrich-provider/pom.xml'],
  ['M', 'bookkeeper-stats-providers/twitter-science-provider/pom.xml'],
  ['M', 'compat-deps/bookkeeper-server-compat-4.0.0/pom.xml'],
  ['M', 'compat-deps/bookkeeper-server-compat-4.1.0/pom.xml'],
  ['M', 'compat-deps/bookkeeper-server-compat-4.2.0/pom.xml'],
  ['M', 'compat-deps/hedwig-server-compat-4.0.0/pom.xml'],
  ['M', 'compat-deps/hedwig-server-compat-4.1.0/pom.xml'],
  ['M', 'compat-deps/hedwig-server-compat-4.2.0/pom.xml'],
  ['M', 'compat-deps/pom.xml'],
  ['M', 'hedwig-client/pom.xml'],
  ['M', 'hedwig-client-jms/pom.xml'],
  ['M', 'hedwig-protocol/pom.xml'],
  ['M', 'hedwig-server/pom.xml'],
  ['M', 'pom.xml']])

In [50]:
info

{'CHANGES.txt': (1, 1, 3, 4),
 'bookkeeper-benchmark/pom.xml': (1, 1, 6, 4),
 'bookkeeper-server/pom.xml': (1, 1, 6, 4),
 'compat-deps/bookkeeper-server-compat-4.0.0/pom.xml': (1, 1, 6, 4),
 'compat-deps/bookkeeper-server-compat-4.1.0/pom.xml': (1, 1, 6, 4),
 'compat-deps/hedwig-server-compat-4.0.0/pom.xml': (1, 1, 6, 4),
 'compat-deps/hedwig-server-compat-4.1.0/pom.xml': (1, 1, 6, 4),
 'compat-deps/pom.xml': (2, 2, 9, 4),
 'hedwig-client/pom.xml': (1, 1, 6, 4),
 'hedwig-protocol/pom.xml': (1, 1, 6, 4),
 'hedwig-server/pom.xml': (1, 1, 6, 4),
 'pom.xml': (1, 1, 6, 4)}

In [66]:
get_commit_info1(repo, commit)

skipping commit: commit b830f4e88c991d67a84ed883c6136989a54c2556
tree dac06a675491b4f71805a4384c22e3456632122d
parent d17c46fc79da130e3738baaa74af9c99f1e234f2
author Sijie Guo <sijie@apache.org> 1423818911 -0800
committer Sijie Guo <sijie@apache.org> 1423818911 -0800

    BookKeeper Release 4.3.1

 , parent: d17c46fc79da130e3738baaa74af9c99f1e234f2 , exception: 'dac06a675491b4f71805a4384c22e3456632122d'


{}